# Import library

In [6]:
import scipy
import csv
import codecs
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold
import random

## Create diff, eval_error, make_features_matrix, pol_reg functions

In [7]:
def diff(a,b):
    if b<1000000:
        while(b<1000000):
            a*=10
            b*=10
    elif b>10000000:
        while(b>10000000):
            b/=10
            a/=10
    return abs(a-b)


def eval_error(size,w,a,y):
    j=0
    for i in range(0,size):
        x=np.transpose(a[i])
        k=y[i]-np.dot(w,x)
        j+=k*k
    return j

def make_features_matrix(deg,size,x1_feature,x2_feature):
    choose=int(((deg+2)*(deg+1))/2)
    a=np.ones((size,choose))
    for i in range(0,size):
        ct=1
        for k in range(0,deg+1):
            for j in range(0,deg+1-k):
                if(j==0 and deg-j-k==0):
                    continue
                a[i][ct]=pow(x1_feature[i],deg-j-k)*pow(x2_feature[i],j)
                ct+=1
    return a

def pol_reg(deg):
    a=make_features_matrix(deg,size_train,x1_train,x2_train)
    b=make_features_matrix(deg,size_test,x1_test,x2_test)
    inv=np.linalg.pinv(a)
    w=np.dot(inv,y_train)
    j_train=eval_error(size_train,w,a,y_train)
    j_test=eval_error(size_test,w,b,y_test)
    print(j_train,j_test)
    with open('results.csv', 'a', newline='') as csv_file:
        writer=csv.writer(csv_file, delimiter=',')
        if deg!=1:
            writer.writerow(['polynomial reg(deg='+str(deg)+')',j_train,j_test])
        elif deg==1:
            writer.writerow(['linear regression',j_train,j_test])
    return w


# Create Gradian Descent function

In [8]:
def gradian_descent(deg,w,eta,intv):
    w_copy=w
    j_main=eval_error(size_train,w,make_features_matrix(deg,size_train,x1_train,x2_train),y_train)
    choose=int(((deg+2)*(deg+1))/2)
    for i in range(len(w)):
        w[i]=w[i]+random.choice([intv*-1,intv])
    start=time.time()
    while(True):
        sigma=np.zeros(choose)
        a=make_features_matrix(deg,size_train,x1_train,x2_train)
        for i in range(0,size_train):
            x=np.transpose(a[i])
            sigma=np.add(np.dot((y_train[i]-(np.dot(w,x)))*eta,np.transpose(x)),sigma)
        w=np.add(w,sigma)
        j_train=eval_error(size_train,w,a,y_train)
        print("J",j_train)
        print("W",w)
        if abs(j_train-j_main)<0.01 or diff(j_train,j_main)<100:
            end=time.time()
            j_test=eval_error(size_test,w,make_features_matrix(deg,size_test,x1_test,x2_test),y_test)
            with open('results.csv', 'a', newline='') as csv_file:
                writer=csv.writer(csv_file, delimiter=',')
                if deg!=1:
                    writer.writerow([eta,'polynomial reg(deg='+str(deg)+')',end-start,j_train,j_test])
                elif deg==1:
                    writer.writerow([eta,'linear regression',end-start,j_train,j_test])
            break

# Make a Five Fold function 

In [9]:
def fiveFold(deg):
    ls=[]
    for i in range(0,size_train):
        ls.append(i)
    kf=KFold(n_splits=5,shuffle=True)
    lamb=[10000,1000,100,10,1,0.1,0.01,0.001,0.0001]
    rec=[]
    for dah in lamb:
        er=0
        w_hat=[]
        for train_set,test_set in kf.split(ls):
            y_hat_train=[]
            x1_hat_train=[]
            x2_hat_train=[]
            for i in range(0,len(train_set)):
                x1_hat_train.append(x1_train[train_set[i]])
                x2_hat_train.append(x2_train[train_set[i]])
                y_hat_train.append(y_train[train_set[i]])
            a=make_features_matrix(deg,len(train_set),x1_hat_train,x2_hat_train)

            y_hat_test=[]
            x1_hat_test=[]
            x2_hat_test=[]
            for i in range(0,len(test_set)):
                x1_hat_test.append(x1_train[test_set[i]])
                x2_hat_test.append(x2_train[test_set[i]])
                y_hat_test.append(y_train[test_set[i]])
            b=make_features_matrix(deg,len(test_set),x1_hat_test,x2_hat_test)

            a_c=np.dot(np.transpose(a),a)
            for i in range(0,3):
                a_c[i][i]+=dah
            inv=np.dot(np.linalg.inv(a_c),np.transpose(a))
            w_hat=np.dot(inv,y_hat_train)

            j_train=eval_error(len(test_set),w_hat,b,y_hat_test)
            er+=j_train
        print("err",er/5,"lambda",dah)
        b=make_features_matrix(deg,size_test,x1_test,x2_test)
        j_test=eval_error(size_test,w_hat,b,y_test)
        rec.append((er/5,dah,j_test))
    rec.sort()
    with open('results.csv', 'a', newline='') as csv_file:
        writer=csv.writer(csv_file, delimiter=',')
        writer.writerow(['lambda','method','the test fold error(avg)','actual test_set error'])
        for i in range(0,len(rec)):
            if i==0:
                if deg!=1:
                    writer.writerow(['*'+str(rec[i][1])+'(min err)','polynomial reg(deg='+str(deg)+')',rec[i][0],rec[i][2]])
                elif deg==1:
                    writer.writerow(['*'+str(rec[i][1])+'(min err)','linear regression',rec[i][0],rec[i][2]])
            else:
                if deg!=1:
                    writer.writerow([rec[i][1],'polynomial reg(deg='+str(deg)+')',rec[i][0],'-'])
                elif deg==1:
                    writer.writerow([rec[i][1],'linear regression',rec[i][0],'-'])


# Main

In [10]:
if __name__ == "__main__":
    #Reading the data and the stuff
    df=np.load('data.npz')
    y_train=df['y']
    x1_train=df['x1']
    x2_train=df['x2']
    y_test=df['y_test']
    x1_test=df['x1_test']
    x2_test=df['x2_test']
    size_train=len(x1_train)
    size_test=len(x1_test)
    #Preparin the csv output
    with open('results.csv', 'w', newline='') as csv_file:
        writer=csv.writer(csv_file, delimiter=',')
        writer.writerow(['Without Normalization'])
        writer.writerow(['method','train error', 'test error'])
    #some intializations
    degrees=[1,3,5]
    weights=[]
    etas=[0.0000016,0.00000000001,0.0000000000000001]
    interval=[100,0.00001,0.0000001]
    #the a) part of the exercise
    for deg in degrees:
        weights.append(pol_reg(deg))

    with open('results.csv', 'a', newline='') as csv_file:
        writer=csv.writer(csv_file, delimiter=',')
        writer.writerow('---')
        writer.writerow(['Gradian Descent'])
        writer.writerow(['eta','method','elapsed time(s)','train error','test error'])
    #the b part of the exercise
    for i in range(len(degrees)):
        gradian_descent(degrees[i],weights[i],etas[i],interval[i])

    with open('results.csv', 'a', newline='') as csv_file:
        writer=csv.writer(csv_file, delimiter=',')
        writer.writerow('---')
        writer.writerow(['With Normalization'])
    #the c) part of the exercise
    for deg in degrees:
        fiveFold(deg)


18317759690.368008 12944759955.772049
1.2445274406972037e-18 7.060354541745272e-19
5.200849019904746e-17 3.195696115912163e-17
J 40911658897.54218
W [-4305.57980531   550.29885392   593.4194832 ]
J 39820420713.96518
W [-4324.16720329   456.27838717   345.39405675]
J 38798650206.504005
W [-4304.76131985   561.81632287   581.95262492]
J 37837162893.638626
W [-4322.63254575   470.11866416   346.00437987]
J 36929016311.772766
W [-4304.21324117   569.24047347   572.50584236]
J 36068855923.233025
W [-4321.360264     480.34879194   347.83911951]
J 35252457861.25911
W [-4303.84889445   573.83731091   564.53347925]
J 34476408994.44794
W [-4320.27632428   488.03470937   350.46360218]
J 33737882802.34541
W [-4303.6080682    576.48847884   557.65202788]
J 33034482113.14852
W [-4319.32886709   493.92098343   353.57760972]
J 32364128513.110386
W [-4303.44899514   577.80763295   551.59119419]
J 31724984345.376057
W [-4318.48150707   498.52800959   356.97455748]
J 31115397475.732147
W [-4303.34276036 

J 18425160425.074337
W [-4295.51374111   546.85962984   457.9917152 ]
J 18420514242.592087
W [-4296.69091024   539.98706407   441.44543867]
J 18416078178.606415
W [-4295.29572295   546.68053513   457.60131341]
J 18411842696.881313
W [-4296.41425671   540.11952396   441.80599497]
J 18407798694.103767
W [-4295.0772249    546.50866989   457.22819282]
J 18403937480.228638
W [-4296.13980364   540.24507075   442.14974074]
J 18400250759.718174
W [-4294.85830546   546.34370911   456.87157047]
J 18396730613.6306
W [-4295.86748775   540.36402146   442.47744071]
J 18393369482.524525
W [-4294.63902031   546.18534251   456.5306989 ]
J 18390160150.136868
W [-4295.59724852   540.47667872   442.78982489]
J 18387095727.80407
W [-4294.4194225    546.03327388   456.2048646 ]
J 18384169639.588737
W [-4295.32902806   540.5833314    443.08759016]
J 18381375608.082767
W [-4294.19956254   545.88722047   455.89338646]
J 18378707640.854504
W [-4295.06277094   540.68425525   443.37140174]
J 18376160017.5112
W [-

J 18321795500.87868
W [-4282.9074095    542.4645193    449.60090377]
J 18321763484.935295
W [-4282.89066709   542.0132542    448.52986667]
J 18321732387.775723
W [-4282.70819484   542.43689058   449.56783088]
J 18321702169.5792
W [-4282.68799864   542.00574641   448.54523778]
J 18321672792.32609
W [-4282.50960398   542.40978496   449.53590038]
J 18321644219.716217
W [-4282.4861257    541.99784954   448.55955263]
J 18321616417.090496
W [-4282.31163839   542.38318127   449.50506169]
J 18321589351.357086
W [-4282.28504187   541.98958383   448.57286045]
J 18321562990.919506
W [-4282.11429934   542.3570593    449.47526651]
J 18321537305.609344
W [-4282.08474095   541.98096862   448.58520821]
J 18321512266.62118
W [-4281.91758796   542.33139975   449.44646873]
J 18321487846.450806
W [-4281.8852169    541.97202232   448.59664076]
J 18321464018.83646
W [-4281.72150522   542.30618419   449.41862431]
J 18321440758.701405
W [-4281.68646387   541.96276252   448.6072009 ]
J 18321418042.101856
W [-4

J 18320130886.11205
W [-4272.14263886   541.28785822   448.56637454]
J 18320122430.502724
W [-4272.05582152   541.30866169   448.63011357]
J 18320114007.9928
W [-4271.97960234   541.27435105   448.56097087]
J 18320105618.320286
W [-4271.89331336   541.29391368   448.62167068]
J 18320097261.230312
W [-4271.81714745   541.26086483   448.55552169]
J 18320088936.475296
W [-4271.73137492   541.27924401   448.61332078]
J 18320080643.814762
W [-4271.65527189   541.24740072   448.55003009]
J 18320072383.014397
W [-4271.57000438   541.26465121   448.60506067]
J 18320064153.846638
W [-4271.49397339   541.23395979   448.54449904]
J 18320055956.089653
W [-4271.40919997   541.25013388   448.5968873 ]
J 18320047789.527485
W [-4271.33324972   541.22054308   448.53893134]
J 18320039653.94981
W [-4271.24895989   541.23569067   448.58879775]
J 18320031549.151382
W [-4271.17309862   541.20715155   448.53332968]
J 18320023474.93207
W [-4271.08928233   541.22132028   448.58078923]
J 18320015431.096676
W [-

J 18319319667.080635
W [-4263.45079563   540.54647428   448.22764262]
J 18319314150.307667
W [-4263.38430242   540.54252789   448.22915651]
J 18319308653.056606
W [-4263.31860046   540.53503649   448.22210533]
J 18319303175.25798
W [-4263.25235666   540.53102877   448.2234324 ]
J 18319297716.84248
W [-4263.18687309   540.52363742   448.21658345]
J 18319292277.740803
W [-4263.12087717   540.51957201   448.2177326 ]
J 18319286857.8842
W [-4263.05561185   540.512277     448.21107712]
J 18319281457.204273
W [-4262.98986232   540.50815738   448.21205684]
J 18319276075.632492
W [-4262.92481509   540.50095519   448.20558646]
J 18319270713.101032
W [-4262.85931048   540.49678465   448.20640487]
J 18319265369.54199
W [-4262.79448114   540.48967191   448.2001116 ]
J 18319260044.88796
W [-4262.72922003   540.4854536    448.20077642]
J 18319254739.071545
W [-4262.66460836   540.47842709   448.19465263]
J 18319249452.02588
W [-4262.59958935   540.47416403   448.19517126]
J 18319244183.684284
W [-42

...

J 0.010492073335924503
W [ 9.99999900e-01 -1.47028086e-09 -2.88292102e-08 -5.19873845e-08
  9.56251930e-08 -2.33847421e-08 -3.74810722e-09 -8.94081914e-08
 -9.27163682e-08 -9.59530923e-08 -1.02405653e-07  9.75636451e-08
 -9.88588265e-08 -9.92659410e-08  4.00000010e+00  1.00223609e-07
 -9.98767323e-08  1.00073497e-07  2.00000010e+00  3.00000010e+00
 -9.99890885e-08]
J 0.010486218777346725
W [ 9.99999900e-01 -1.45591873e-09 -2.88213590e-08 -5.19849836e-08
  9.56217061e-08 -2.33864140e-08 -3.74764808e-09 -8.94066173e-08
 -9.27154956e-08 -9.59527105e-08 -1.02405393e-07  9.75629418e-08
 -9.88586519e-08 -9.92658398e-08  4.00000010e+00  1.00223547e-07
 -9.98767126e-08  1.00073509e-07  2.00000010e+00  3.00000010e+00
 -9.99890879e-08]
J 0.010480372900977175
W [ 9.99999900e-01 -1.44156642e-09 -2.88135154e-08 -5.19825891e-08
  9.56182161e-08 -2.33880796e-08 -3.74718984e-09 -8.94050443e-08
 -9.27146237e-08 -9.59523293e-08 -1.02405132e-07  9.75622378e-08
 -9.88584773e-08 -9.92657387e-08  4.00000010

J 0.010359580941786592
W [ 9.99999900e-01 -1.14241703e-09 -2.86505103e-08 -5.19337474e-08
  9.55442217e-08 -2.34216044e-08 -3.73777237e-09 -8.93722538e-08
 -9.26964920e-08 -9.59444530e-08 -1.02399689e-07  9.75473135e-08
 -9.88548382e-08 -9.92636333e-08  4.00000010e+00  1.00222181e-07
 -9.98762822e-08  1.00073775e-07  2.00000010e+00  3.00000010e+00
 -9.99890761e-08]
J 0.010353921213852839
W [ 9.99999900e-01 -1.12827806e-09 -2.86428289e-08 -5.19314896e-08
  9.55406650e-08 -2.34231325e-08 -3.73733360e-09 -8.93707037e-08
 -9.26956369e-08 -9.59440841e-08 -1.02399431e-07  9.75465961e-08
 -9.88546661e-08 -9.92635338e-08  4.00000010e+00  1.00222118e-07
 -9.98762628e-08  1.00073787e-07  2.00000010e+00  3.00000010e+00
 -9.99890756e-08]
J 0.01034826973004932
W [ 9.99999900e-01 -1.11414866e-09 -2.86351547e-08 -5.19292379e-08
  9.55371053e-08 -2.34246544e-08 -3.73689570e-09 -8.93691547e-08
 -9.26947826e-08 -9.59437157e-08 -1.02399173e-07  9.75458782e-08
 -9.88544942e-08 -9.92634345e-08  4.00000010e

J 0.010231460644783363
W [ 9.99999900e-01 -8.19626130e-10 -2.84756606e-08 -5.18833484e-08
  9.54616756e-08 -2.34552100e-08 -3.72789890e-09 -8.93368619e-08
 -9.26770128e-08 -9.59361056e-08 -1.02393777e-07  9.75306649e-08
 -9.88509087e-08 -9.92613659e-08  4.00000010e+00  1.00220711e-07
 -9.98758386e-08  1.00074048e-07  2.00000010e+00  3.00000010e+00
 -9.99890644e-08]
J 0.01022598593603328
W [ 9.99999900e-01 -8.05704916e-10 -2.84681440e-08 -5.18812289e-08
  9.54580519e-08 -2.34565989e-08 -3.72747986e-09 -8.93353353e-08
 -9.26761747e-08 -9.59357492e-08 -1.02393521e-07  9.75299340e-08
 -9.88507392e-08 -9.92612682e-08  4.00000010e+00  1.00220646e-07
 -9.98758194e-08  1.00074060e-07  2.00000010e+00  3.00000010e+00
 -9.99890639e-08]
J 0.010220519057306933
W [ 9.99999900e-01 -7.91793049e-10 -2.84606346e-08 -5.18791154e-08
  9.54544254e-08 -2.34579818e-08 -3.72706166e-09 -8.93338097e-08
 -9.26753374e-08 -9.59353933e-08 -1.02393266e-07  9.75292026e-08
 -9.88505698e-08 -9.92611706e-08  4.00000010e

J 0.010107492915029817
W [ 9.99999900e-01 -5.01785798e-10 -2.83045510e-08 -5.18360815e-08
  9.53776184e-08 -2.34856650e-08 -3.71847229e-09 -8.93020027e-08
 -9.26579195e-08 -9.59280411e-08 -1.02387916e-07  9.75137096e-08
 -9.88470367e-08 -9.92591378e-08  4.00000010e+00  1.00219199e-07
 -9.98754011e-08  1.00074318e-07  2.00000010e+00  3.00000010e+00
 -9.99890532e-08]
J 0.010102193967525534
W [ 9.99999900e-01 -4.88077104e-10 -2.82971946e-08 -5.18340959e-08
  9.53739304e-08 -2.34869192e-08 -3.71807236e-09 -8.93004990e-08
 -9.26570979e-08 -9.59276968e-08 -1.02387662e-07  9.75129656e-08
 -9.88468697e-08 -9.92590418e-08  4.00000010e+00  1.00219132e-07
 -9.98753822e-08  1.00074330e-07  2.00000010e+00  3.00000010e+00
 -9.99890528e-08]
J 0.010096902459532247
W [ 9.99999900e-01 -4.74377533e-10 -2.82898451e-08 -5.18321160e-08
  9.53702397e-08 -2.34881678e-08 -3.71767324e-09 -8.92989962e-08
 -9.26562770e-08 -9.59273529e-08 -1.02387409e-07  9.75122210e-08
 -9.88467027e-08 -9.92589459e-08  4.00000010

err 3670555657.4510484 lambda 10
err 3667516525.3793535 lambda 1
err 3666221390.778351 lambda 0.1
err 3667010232.8248115 lambda 0.01
err 3668252734.875988 lambda 0.001
err 3670537194.4957123 lambda 0.0001
err 20.226817870048098 lambda 10000
err 17.561206634509695 lambda 1000
err 6.207812725299476 lambda 100
err 0.24503369042893355 lambda 10
err 0.0029945766539795507 lambda 1
err 3.05561045407491e-05 lambda 0.1
err 3.066662895380354e-07 lambda 0.01
err 3.103341520810731e-09 lambda 0.001
err 3.0647244258702666e-11 lambda 0.0001
err 4.196046530421456 lambda 10000
err 4.0841360806675135 lambda 1000
err 3.114568100704892 lambda 100
err 0.6033000860457054 lambda 10
err 0.013488912967294698 lambda 1
err 0.00014959123320223728 lambda 0.1
err 1.5330868824129439e-06 lambda 0.01
err 1.6970348644000454e-08 lambda 0.001
err 3.508178107540651e-09 lambda 0.0001
